
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Głębokie sieci warstwowe i regularyzacja</big></big></big></big></big>

---

In [3]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

#from bokeh.io import gridplot, output_file, show
#from bokeh.plotting import figure, output_notebook
#from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

# Głębokie sieci
> __Ogólne twierdzenie o aproksymacji__ mówi, że sieć warstwowa, z liniowymi neuronami ukrytymi oraz jedną warstwą ukrytą ze _zgniatającą_ funkcją aktywacji (np. logistyczną) jest w stanie aproksymować z dowolną dokładnością dowolną mierzalną funkcję ciągłą na domknietym podzbiorze $\mathbb{R}^n$ pod warunkiem, że sieć będzie miała __wystarczającą__ liczbe neuronów ukrytych.

* liczba neuronów ukrytych jest zwykle wykładniczo duża
* sieć będzie potrafiła __aproksymować__ funkcję, ale nie ma żadnej gwarancji, że się jej __nauczy__
* ale istnieje rodzina funkcji, które mogą być przedstawione pod warunkiem odpowiedniej __głębokości__
* empirycznie wiemy, że większa głębokość daje zwykle lepszą generalizację

# Regularyzacja
Celem regularyzacji jest osiągniecie modelu o niższej złożoności.
* zwykle celem jest minimalizacja __wariancji__ kosztem nieco zwiększonego __biasu__
* reguła Ockhama
  * jest bardziej prawdopodobne, że model prostszy jest prawdziwy, niż model bardziej złożony
    * spośród dwóch modeli tłumaczących równie dobrze jakieś zjawisko wybrać ten prostszy
  * optymalizacja parametrów $\theta$
  $$\begin{align}
  \theta^\ast&=\underset{\theta}{\arg\max}P(D\mid\theta)P(\theta)=\underset{\theta}{\arg\max}\log\,P(D\mid\theta)P(\theta)\\
  &=\underset{\theta}{\arg\max}[\log\,P(D\mid\theta)+\log\,P(\theta)]\\
  &=\underset{\theta}{\arg\min}[-\log\,P(D\mid\theta)-\log\,P(\theta)]
  \end{align}$$
    * czynnik $-\log\,P(\theta)$ odpowiada złożoności zestawu parametrów
    * to reguła Minimum Description Length: najlepszą hipotezą dla z
  * jest wiele poglądów, że powoływanie się na regułę Ockhama jest fałszywe

## Regularyzacja przez czynnik kary
$$L(w; X, y)=L(w; X, y)+\alpha\Omega(w)$$
* algorytm uczący bedzie minimalizował koszt na danych
* oraz jakąś miarę na parametrach $w$
  * __nie__ regularyzujemy biasów
  * wagi kontrolują interakcję między dwoma neuronami
  * biasy tylko jednego neuronu
  * biasy wprowadzają mniej wariancji w modelu
* może się okazać potrzebne __oddzielne__ regularyzowanie różnych warstw
  * różne $\alpha$ dla róznych warstw

## $L2$
$$\Omega(w)=\frac{1}{2}\|w\|_2^2$$
* ridge regression albo regresja Tikhonova
$$\begin{align}
w&=w-\epsilon(\alpha{}w+\nabla_wL(w))\\
&=(1-\epsilon\alpha)w-\epsilon\nabla_wL(w)
\end{align}$$

Regularyzacja L2 powoduje __skalowanie__ optymalnego wektora wag $w^*$ __wzdłuż__ wektorów głównych zdefiniowanych przez macierz Hesjanu $H$
<img src="../nn_figures/L2.pdf" width="80%"> [Goodfellow et al.]
1. wektor $w^*$ jest przeskalowany wzdłuż $i$-tego wektora głównego $H$ przez czynnik $\dfrac{\lambda_i}{\lambda_i+\alpha}$, gdzie $\alpha$ jest czynnikiem  funkcji kary
  * jeśli $\lambda_i>>\alpha$, to regularyzacja będzie niewielka
  * jeśli $\lambda_i<<\alpha$, to współrzędna będzie zredukowana prawie do minimum
2. na rysunku
  * $\tilde{\ w}$ jest punktem ekwilibrium miedzy minimum kosztu, a regularyzacją
  * w pierwszym głównym kierunku wzdłuż $w_1$ wartość własna $H$ jest niewielka
    * funkcja kosztu nie zmienia się wiele w tym kierunku, a wobec tego regularyzator ma duży wpływ i skraca $w_1$ do zera
  * w drugim kierunku funkcja celu jest bardzo czula na zmiany
    * wartość własna jest wysoka
    * regularyzator mało wpływa

## $L1$
Składnik kosztu jako $$\Omega(w)=\|w\|_1=\sum_i\mid w_i\mid$$
* tylko wagi, nie bias
* gradient $$\nabla_wL(w)=\nabla_wL(w)+\alpha{}sign(w)$$
* zachowanie jest inne niż $L2$
  * gradient nie skaluje się liniowo z każdym $w$, stąd brak czystych rozwiązań
  * rozwinięcie funkcji kosztu wraz z poprawką $L1$ 
  $$L(w)=L(w+(w-w^*))=L(w)+\sum_i\left[\frac{1}{2}H(w-w^*)+\alpha\mid{}w_i\mid\right]$$
    * skladnik pierwszej pochodnej znika, bo $w^*$ jest optimum
  * rozwiązanie analityczne ma postać
  $$w_i=sign(w_i^*)\,\max\left[\mid{}w_i^*\mid-\frac{\alpha}{H_{i,i}},\,0\right]$$
  zakładając, że $H$ jest diagonalna
    * jeśli $\mid{}w_i^*\mid\leq\frac{\alpha}{H_{i,i}}$
      * tu rozwiązaniem jest $w_i=0$
      * regularyzacja przeważa
    * jeśli $\mid{}w_i^*\mid>\frac{\alpha}{H_{i,i}}$
      * $w_i$ jest przesuwane
* $L1$ daje bardziej __rzadkie__ rozwiązanie


## Rozszerzanie zbioru uczącego
1. na pewno lepiej jest mieć więcej danych niż mniej
2. sztuczne dane
  * klasyfikator powinien być inwariantny na transformacje - wystarczy je zastosować
  * ale ostrożnie gdy mogą one zmienić znaczenie: 6 a 9
3. szum losowy
  * także dodawany do neuronów ukrytych
  * sieci bywają mało odporne na szum
  * wykorzystywane w tzw. __denoising autoencoders__
  * także __dropout__ jest formą dodawania szumu (właściwie __mnożenia__ przez szum)
4. ostrożność przy porównywaniu algorytmów
  * jeden algorytm działa słabo przy danych czystych
  * drugi dobrze przy danych rozszerzonych
  * jaki można wyciągnąć wniosek?
  * ale pamiętac by warunki porównania były sprawiedliwe (ten sam zbiór dla porównań)

## Dodawanie szumu do neuronów wyjsciowych
* zbiory mają błędy w etykietach
* maksymalizacja $\log\,p(y\mid x)$ może zaszkodzić, jesli etykieta $y$ jest błędna
* jak temu zaradzić?
  * modelować szum etykietowań
  * załóżmy, że etykiety są poprawne z prawdopodobieństwem $1-\epsilon$
  * najlepiej __dodać__ to założenie do funkcji kosztu
  * __label smoothing__
    * zastąpienie w modelu opartym o _softmax_ etykiet $0$ i $1$ przez $\frac{\epsilon}{k}$ i $1-\frac{k-\epsilon}{k}$
    * uczenie nigdy nie dotrze do etykiet $0$ i $1$
    * będzie się długo uczyć ciągle __powiekszając__ wagi
    * konieczne dodanie jakiegoś _weight decay_